In [93]:
import numpy as pd
import pandas as pd
%pylab inline
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Populating the interactive namespace from numpy and matplotlib


In [94]:
def read_data():
    emails = pd.read_csv('./Emails.csv').dropna(subset=['SenderPersonId','ExtractedBodyText','ExtractedSubject'])
    emails_receivers = pd.read_csv('EmailReceivers.csv')
            
    text = [re.sub('[^a-zA-Z0-9]', ' ', bodytext) for bodytext in emails.ExtractedBodyText]
    return pd.DataFrame({'sender': list(map(int, emails.SenderPersonId)), 
                         'subject': emails.ExtractedSubject, 'message': text})

In [95]:
df = read_data()
df.head()

,message,sender,subject
2,Thx,32,Re: Chris Stevens
5,Pis print H hrod17 clintonernail...,80,Meet The Right Wing Extremist Behind Anti-Musl...
8,FYI,87,FVV: Secretary's remarks
10,Fyi B6,87,AbZ and Hb3 on Libya and West Bank/Gaza
12,Fyi,87,hey


Ngrams

In [96]:
bigram_vectorizer = CountVectorizer(ngram_range=(2,2), min_df=10).fit(df.message)

In [133]:
sorted(bigram_vectorizer.vocabulary_.items(), key=lambda v:-v[1])[:10]

[(u'your trip', 3168),
 (u'your own', 3167),
 (u'your message', 3166),
 (u'your help', 3165),
 (u'your call', 3164),
 (u'you would', 3163),
 (u'you will', 3162),
 (u'you when', 3161),
 (u'you were', 3160),
 (u'you we', 3159)]

Collocations

In [98]:
import nltk.collocations
from nltk.tokenize import word_tokenize

bam = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(word_tokenize(' '.join(df.message)))
finder.apply_freq_filter(4)
finder.nbest(bam.pmi, 10)

[('DENNIS', 'ROSS'),
 ('LUIS', 'AMADO'),
 ('MARIE', 'SLAUGHTER'),
 ('NOER', 'HASSAN'),
 ('Vital', 'Voices'),
 ('x7', '7203'),
 ('ANNE', 'MARIE'),
 ('ARTURO', 'VALENZUELA'),
 ('BINYAMIN', 'NETANYAHU'),
 ('Beacon', 'Theater')]

Clusterisation

In [120]:
X = TfidfVectorizer(max_df=1000, min_df=10).fit_transform(df.message).toarray()

In [126]:
from sklearn.cluster import KMeans
CLUSTERS_COUNT = 7

predicts = KMeans(n_clusters=CLUSTERS_COUNT, random_state=1).fit_predict(X)

In [127]:
df_predicts = df.copy()
df_predicts['predicts'] = predicts

In [128]:
df_preds.head()

,message,sender,subject,preds
2,Thx,32,Re: Chris Stevens,7
5,Pis print H hrod17 clintonernail...,80,Meet The Right Wing Extremist Behind Anti-Musl...,5
8,FYI,87,FVV: Secretary's remarks,8
10,Fyi B6,87,AbZ and Hb3 on Libya and West Bank/Gaza,8
12,Fyi,87,hey,8


In [131]:
for cluster in xrange(CLUSTERS_COUNT):
    cv = CountVectorizer(stop_words='english').fit(df_predicts[df_predicts.predicts == cluster].message)
    word_counts = sorted(cv.vocabulary_.items(), key=lambda v:-v[1])[:5]
    print cluster, word_counts

0 [(u'zuckerman', 5886), (u'zone', 5885), (u'zia', 5884), (u'zi', 5883), (u'zelaya', 5882)]
1 [(u'sigh', 11), (u'paymasters', 10), (u'just', 9), (u'jpg', 8), (u'image003', 7)]
2 [(u'yup', 117), (u'yes', 116), (u'yep', 115), (u'xo', 114), (u'works', 113)]
3 [(u'zzity', 20609), (u'zx', 20608), (u'zwire', 20607), (u'zurich', 20606), (u'zumbi', 20605)]
4 [(u'vu', 60), (u'version', 59), (u'thx', 58), (u'speech', 57), (u'review', 56)]
5 [(u'zimbabwean', 2642), (u'zia', 2641), (u'zebari', 2640), (u'zatlers', 2639), (u'yukiya', 2638)]
6 [(u'yes', 25), (u'work', 24), (u'waldorf', 23), (u'thx', 22), (u'talk', 21)]


In [132]:
for cluster in xrange(CLUSTERS_COUNT):
    cv = CountVectorizer(stop_words='english').fit(df_predicts[df_predicts.predicts == cluster].subject)
    word_counts = sorted(cv.vocabulary_.items(), key=lambda v:-v[1])[:5]
    print cluster, word_counts

0 [(u'zuckerman', 2373), (u'zest', 2372), (u'zelaya', 2371), (u'zebari', 2370), (u'zak', 2369)]
1 [(u'zelikow', 970), (u'zelaya', 969), (u'zardari', 968), (u'young', 967), (u'yohannes', 966)]
2 [(u'young', 208), (u'wrong', 207), (u'work', 206), (u'wilson', 205), (u'weekend', 204)]
3 [(u'zelaya', 1492), (u'zebari', 1491), (u'zardari', 1490), (u'zach', 1489), (u'youtube', 1488)]
4 [(u'zones', 282), (u'york', 281), (u'women', 280), (u'week', 279), (u'ways', 278)]
5 [(u'zebari', 566), (u'zealand', 565), (u'yohannes', 564), (u'yellow', 563), (u'yang', 562)]
6 [(u'zones', 262), (u'yohannes', 261), (u'workers', 260), (u'work', 259), (u'word', 258)]


По поводу способа интерпретировать результаты:
Посадить ассессора и для каждого слова из небольшой случайной подвыборки кластера отвечать на вопросы: 
* принадлежит ли слово оставшейся части кластера (на его взгляд)
* отличается ли слово от остальных кластеров

И можно строить аналоги precision и recall